Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [21]:
print(aa)

input


In [10]:
# generator version
def fibon(n):
    a = b = 1
    for i in xrange(n):
        yield a
        a, b = b, a + b

for x in fibon(10):
    print x,

1 1 2 3 5 8 13 21 34 55


In [15]:
''' generator, ref: http://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do'''

filter_sizes = [1,2]
feature_maps = [3,4]
conv_parameters = zip(filter_sizes, feature_maps)
c = (a+b for i, (a, b) in enumerate(conv_parameters))

In [16]:
list(c)

[4, 6]

In [72]:
'''class hierarchy, super'''
class A(object):
    def __init__(self,**kwargs):
        print("A.__init__()")
#         print(kwargs['c'])

class B(A):
    def __init__(self,**kwargs):
        print("B.__init__()")
        for name, value in kwargs.items():
            print(value)
        super(B, self).__init__(**kwargs)

class C(A):
    def __init__(self,**kwargs):
        print("C.__init__()")
        for name, value in kwargs.items():
            print(value)
        super(C,self).__init__(**kwargs)
        
class D(B,C):
    def __init__(self,a,**kwargs):
        print("init D")
        super(D,self).__init__(**kwargs)
d = D(a=0,b=1,c=2)

init D
B.__init__()
C.__init__()
A.__init__()


In [80]:
'''decorator'''
def entryExit(f):
    def new_f():
        print "Entering", f.__name__
        f()
        print "Exited", f.__name__
    return new_f

@entryExit
def func1():
    print "inside func1()"

@entryExit
def func2():
    print "inside func2()"
func1()
func2.__name__

Entering func1
inside func1()
Exited func1


'new_f'

In [130]:
'''decorator inside a class for class method'''
class Test(object):
    def _decorator(foo):
        def magic( self ) :
            print "start magic"
            foo( self )
            print "end magic"
        return magic

    @_decorator
    def bar( self ) :
        print "normal call"

test = Test()

test.bar()

start magic
normal call
end magic


In [129]:
'''decorator outside class'''
class MyDec(object):
    def __init__(self,flag):
        self.flag = flag
    def __call__(self, original_func):
        decorator_self = self
        def wrappee( *args, **kwargs):
            print 'in decorator before wrapee with flag ',decorator_self.flag
            original_func(*args,**kwargs)
            print 'in decorator after wrapee with flag ',decorator_self.flag
        return wrappee

@MyDec('foo de fa fa')
def bar(a,b,c):
    print 'in bar',a,b,c
bar('x','y','z')

in decorator before wrapee with flag  foo de fa fa
in bar x y z
in decorator after wrapee with flag  foo de fa fa


In [132]:
'''decorator inside a class for a function'''
def p_decorate(func):
    def func_wrapper(self):
        return "<p>{0}</p>".format(func(self))
    return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

my_person = Person()
print my_person.get_fullname()

<p>John Doe</p>


In [183]:
'''conv3d in theano and cudnn'''

%reset
import theano
import theano.tensor as T
from theano.sandbox.cuda.dnn import dnn_conv, dnn_conv3d
from theano.tensor.nnet import conv2d
import numpy as np

theano.config.floatX = 'float32'
floatX = theano.config.floatX

print(theano.config.floatX)

conv_dim = 2
if conv_dim==2:
    dtensor4 = T.TensorType(floatX, (False,)*4)
    img = dtensor4(name='input')
    kerns = dtensor4(name='weight')
    base = dnn_conv(img = img, kerns = kerns, border_mode = 'valid', conv_mode = 'conv')        
#     conved = base.sum(1)
    conved = base
    myconv2d = theano.function([img,kerns], conved)
    
    a = np.zeros((4,3,5,5), dtype=floatX)
    a[0,0,:,:] = np.ones((5,5), dtype=floatX)
    a[1,1,:,:] = 3*np.ones((5,5), dtype=floatX)
    b = np.zeros((2,3,3,3), dtype=floatX)
    b[0,0,:,:] = 2*np.ones((3,3), dtype=floatX)
    b[1,1,:,:] = 4*np.ones((3,3), dtype=floatX)

    c = np.asarray(myconv2d(a,b))
    print(c.shape)
    print(c)
    
else:        
    dtensor5 = T.TensorType(floatX, (False,)*5)
    img = dtensor5(name='input')
    kerns = dtensor5(name='weight')
#     dtensor4 = T.TensorType(floatX, (False,)*4)
#     subsample = dtensor4(name='weight')
    base = dnn_conv3d(img = img, kerns = kerns, border_mode = 'valid', conv_mode = 'conv')     
#     conved = base.sum(1)
    conved = base
    conv3d = theano.function([img,kerns], conved)

    a = np.zeros((4,3,5,5,5), dtype=floatX)
    b = np.zeros((4,3,3,3,3), dtype=floatX) # # of channel in next layer, # of channel this layer, volume size
    print(a.dtype)
    print('input size',a.shape)
    print('filter size',b.shape)
    c = np.asarray(conv3d(a,b))
    print('output size',c.shape)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
float32
(4, 2, 3, 3)
[[[[  18.   18.   18.]
   [  18.   18.   18.]
   [  18.   18.   18.]]

  [[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]]


 [[[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]

  [[ 108.  108.  108.]
   [ 108.  108.  108.]
   [ 108.  108.  108.]]]


 [[[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]

  [[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]]


 [[[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]

  [[   0.    0.    0.]
   [   0.    0.    0.]
   [   0.    0.    0.]]]]


In [29]:
'''2d cnn layer in blocks'''
%reset
import numpy as np
import theano
from theano import tensor
from blocks.bricks.base import Brick
from blocks.bricks.conv import (Convolutional, ConvolutionalSequence,Flattener, MaxPooling)
from toolz.itertoolz import interleave
from blocks.initialization import Constant, Uniform, IsotropicGaussian
from blocks.bricks import MLP, Tanh, Softmax
theano.config.floatX = 'float32'
floatX = theano.config.floatX

convnet = Convolutional(
                    filter_size=(3,3),
                    num_filters=2,
                    num_channels=3,
                    step=(1,1),
                    border_mode='valid',
                    weights_init = Constant(0),
                    biases_init=Constant(0),
                    name='conv_0')
convnet.initialize()
x = tensor.tensor4('features')
h = convnet.apply(x)
f = theano.function([x], h)

a = np.zeros((4,3,5,5), dtype=floatX)
a[0,0,:,:] = np.ones((5,5), dtype=floatX)
a[1,1,:,:] = 3*np.ones((5,5), dtype=floatX)
# b = np.zeros((4,3,3,3), dtype=floatX)
# b[0,0,:,:] = 2*np.ones((3,3), dtype=floatX)
# b[1,1,:,:] = 4*np.ones((3,3), dtype=floatX)
c = f(a)
print(c.shape)
print(c)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
(4, 2, 3, 3)
[[[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]


 [[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]

  [[ 0.  0.  0.]
   [ 0.  0.  0.]
   [ 0.  0.  0.]]]]


In [65]:
% reset
'''native 3d cnn brick in blocks'''
import numpy as np
from blocks.bricks.base import application, lazy
from blocks.bricks import Brick
from blocks.bricks import Feedforward, Initializable
from theano.sandbox.cuda.dnn import dnn_conv, dnn_conv3d
from theano.tensor.nnet import conv2d
import theano
import theano.tensor as T
theano.config.floatX = 'float32'
floatX = theano.config.floatX

class Conv3d(Feedforward,Initializable):
#     def __init__(**args):
#         kerns
    @application(inputs=['input1', 'input2'], outputs=['output'])
    def apply(self, input1, input2):
        base = dnn_conv3d(img = input1, kerns = input2, border_mode = 'valid', conv_mode = 'conv')     
        conved = base
        return conved
conv3d = Conv3d()
conv3d.initialize()

# symbolic graph
dtensor5 = T.TensorType(floatX, (False,)*5)
img = dtensor5(name='input')
kerns = dtensor5(name='weight')
h = conv3d.apply(img,kerns)
f = theano.function([img,kerns], h)

# assign number
a = np.zeros((2,3,5,5,5), dtype=floatX)
b = np.zeros((2,3,3,3,3), dtype=floatX) # # of channel in next layer, # of channel this layer, volume size
a[0,0,:,:,:] = np.ones((5,5), dtype=floatX)
a[1,1,:,:,:] = 3*np.ones((5,5,5), dtype=floatX)
b[0,0,:,:,:] = 2*np.ones((3,3,3), dtype=floatX)
b[1,1,:,:,:] = 4*np.ones((3,3,3), dtype=floatX)
print(a.dtype)
print('input size',a.shape)
print('filter size',b.shape)
c = conv3d.apply(a,b)
print(c.eval())

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
float32
input size (2, 3, 5, 5, 5)
filter size (2, 3, 3, 3, 3)
[[[[[  54.   54.   54.]
    [  54.   54.   54.]
    [  54.   54.   54.]]

   [[  54.   54.   54.]
    [  54.   54.   54.]
    [  54.   54.   54.]]

   [[  54.   54.   54.]
    [  54.   54.   54.]
    [  54.   54.   54.]]]


  [[[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]

   [[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]

   [[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]]]



 [[[[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]

   [[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]

   [[   0.    0.    0.]
    [   0.    0.    0.]
    [   0.    0.    0.]]]


  [[[ 324.  324.  324.]
    [ 324.  324.  324.]
    [ 324.  324.  324.]]

   [[ 324.  324.  324.]
    [ 324.  324.  324.]
    [ 324.  324.  324.]]

   [[ 324.  324.  324.]
    [ 324.  324.  324.]
   

In [14]:
% reset
from blocks.bricks.base import application, lazy
from blocks.bricks import Feedforward, Linear
from blocks.initialization import Constant, Uniform, IsotropicGaussian

class ChainOfTwoFeedforward(Feedforward):
    '''Two sequential Feedforward bricks'''
    def __init__(self, brick1, brick2, **kwargs):
        self.brick1 = brick1
        self.brick2 = brick2
        children = [self.brick1, self.brick2]
        kwargs.setdefault('children', []).extend(children)
        super(Feedforward, self).__init__(**kwargs)

    @property
    def input_dim(self):
        return self.brick1.input_dim

    @input_dim.setter
    def input_dim(self, value):
        self.brick1.input_dim = value

    @property
    def output_dim(self):
        return self.brick2.output_dim

    @output_dim.setter
    def output_dim(self, value):
        self.brick2.output_dim = value

    def _push_allocation_config(self):
        self.brick2.input_dim = self.brick1.get_dim('output')

    @application
    def apply(self, x):
        return self.brick2.apply(self.brick1.apply(x))
    
brick1 = Linear(input_dim=3, output_dim=2, use_bias=False,weights_init=Constant(2))
brick2 = Linear(output_dim=4, use_bias=False, weights_init=Constant(2))
seq = ChainOfTwoFeedforward(brick1, brick2)
seq.initialize()
brick2.input_dim

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


2

In [140]:
'''simple RNN in blocks'''
import numpy
import theano
from theano import tensor
from blocks import initialization
from blocks.bricks import Identity
from blocks.bricks.recurrent import SimpleRecurrent

x = tensor.tensor3('x')
rnn = SimpleRecurrent(dim=4, activation=Identity(), weights_init=initialization.Identity())
rnn.initialize()
h = rnn.apply(x)
f = theano.function([x], h)
print(f(numpy.ones((5, 1, 4), dtype=theano.config.floatX)))

[[[ 1.  1.  1.  1.]]

 [[ 2.  2.  2.  2.]]

 [[ 3.  3.  3.  3.]]

 [[ 4.  4.  4.  4.]]

 [[ 5.  5.  5.  5.]]]


In [136]:
'''two layer rnn in blocks'''
import numpy
from theano import tensor
from blocks.bricks import Identity
from blocks import initialization
from blocks.bricks.recurrent import BaseRecurrent, recurrent
from blocks.bricks.recurrent import SimpleRecurrent
class FeedbackRNN(BaseRecurrent):
    def __init__(self, dim, **kwargs):
        super(FeedbackRNN, self).__init__(**kwargs)
        self.dim = dim
        self.first_recurrent_layer = SimpleRecurrent(dim=self.dim, activation=Identity(), name='first_recurrent_layer',weights_init=initialization.Identity())
        self.second_recurrent_layer = SimpleRecurrent(dim=self.dim, activation=Identity(), name='second_recurrent_layer',weights_init=initialization.Identity())
        self.children = [self.first_recurrent_layer,self.second_recurrent_layer]

    @recurrent(sequences=['inputs'], contexts=[],states=['first_states', 'second_states'],outputs=['first_states', 'second_states'])
    def apply(self, inputs, first_states=None, second_states=None):
        first_h = self.first_recurrent_layer.apply(inputs=inputs, states=first_states + second_states, iterate=False)
        second_h = self.second_recurrent_layer.apply(inputs=first_h, states=second_states, iterate=False)
        return first_h, second_h

    def get_dim(self, name):
        return (self.dim if name in ('inputs', 'first_states', 'second_states')
        else super(FeedbackRNN, self).get_dim(name))

x = tensor.tensor3('x')
feedback = FeedbackRNN(dim=3)
feedback.initialize()
first_h, second_h = feedback.apply(inputs=x)
f = theano.function([x], [first_h, second_h])
for states in f(numpy.ones((3, 1, 3), dtype=theano.config.floatX)):
    print(states)

[[[ 1.  1.  1.]]

 [[ 3.  3.  3.]]

 [[ 8.  8.  8.]]]
[[[  1.   1.   1.]]

 [[  4.   4.   4.]]

 [[ 12.  12.  12.]]]


In [39]:
'''save data to hdf5 format, from Max'''

import numpy as np
import h5py
import tarfile, os
import sys
import cStringIO as StringIO
import tarfile
import time
import zlib

PREFIX = 'data/'
SUFFIX = '.npy.z'

class NpyTarReader(object):
    def __init__(self, fname):
        self.tfile = tarfile.open(fname, 'r|')

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def next(self):
        entry = self.tfile.next()
        if entry is None:
            raise StopIteration()
        name = entry.name[len(PREFIX):-len(SUFFIX)]
        fileobj = self.tfile.extractfile(entry)
        buf = zlib.decompress(fileobj.read())
        arr = np.load(StringIO.StringIO(buf))
        return arr, name

    def close(self):
        self.tfile.close()

train_dataset = NpyTarReader('shapenet10_train.tar')
test_dataset = NpyTarReader('shapenet10_test.tar')

train_features = []
train_targets = []
test_features = []
test_targets = []
for index, (array, name) in enumerate(train_dataset):
    if int(name[-3:])==1:
#         train_features.append(array.flatten())
        train_features.append(array.reshape(1,array.shape[0],array.shape[1],array.shape[2]))
        train_targets.append([int(name[0:3])])
for index, (array, name) in enumerate(test_dataset):
    if int(name[-3:]) == 1:
#         test_features.append(array.flatten())
        test_features.append(array.reshape(1,array.shape[0],array.shape[1],array.shape[2]))
        test_targets.append([int(name[0:3])])

train_features = np.array(train_features)
train_targets = np.array(train_targets)-1 #starts from 0
test_features = np.array(test_features)
test_targets = np.array(test_targets)-1
train_n, c, p1, p2, p3 = train_features.shape
test_n = test_features.shape[0]
n = train_n + test_n

f = h5py.File('shapenet10.hdf5', mode='w')
features = f.create_dataset('input', (n, c, p1, p2, p3), dtype='uint8')
targets = f.create_dataset('targets', (n, 1), dtype='uint8')

features[...] = np.vstack([train_features, test_features])
targets[...] = np.vstack([train_targets, test_targets])

features.dims[0].label = 'batch'
features.dims[1].label = 'input'
targets.dims[0].label = 'batch'
targets.dims[1].label = 'index'

from fuel.datasets.hdf5 import H5PYDataset
split_dict = {
    'train': {'input': (0, train_n), 'targets': (0, train_n)},
    'test': {'input': (train_n, n), 'targets': (train_n, n)}}
f.attrs['split'] = H5PYDataset.create_split_array(split_dict)

f.flush()
f.close()

In [41]:
train_targets.shape

(3977, 1)

In [18]:
'''loading fuel dataset'''

from fuel.datasets.hdf5 import H5PYDataset
train_set = H5PYDataset('shapenet110.hdf5', which_sets=('train',))

In [19]:
print(train_set.provides_sources)
handle = train_set.open()
data = train_set.get_data(handle, slice(0, 1))
print((data[0].shape))

(u'input', u'targets')
(1, 32768)


In [7]:
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
batch_size = 100
train_set_stream = DataStream.default_stream(
    train_set, iteration_scheme=ShuffledScheme(
        train_set.num_examples, batch_size))

In [39]:
import theano.tensor as tensor
A = tensor.zeros((3200, 3, 32, 1))
B = tensor.zeros((3200))
print(A.broadcastable,B.broadcastable)
# B = B.transpose([0, 2, 1])
# A.dimshuffle([0,1,2,'x']) * B.dimshuffle([0,'x',1,2])     
C=(A*B).eval()
C.shape

((False, False, False, True), (False,))


(3200, 3, 32, 3200)

In [86]:
'''upsample in post-process'''
import numpy as np
import scipy.ndimage as ndimage
inarr = np.ones((32,32,32))
def upsample(X,d):
    XX = np.zeros((X.shape[0],X.shape[1],d*X.shape[2],d*X.shape[3],d*X.shape[4]))
    for i,xi in enumerate(X):
        for j,xij in enumerate(xi):
            tmp = (ndimage.zoom(xij, d))
            XX[i,j,:,:,:] = tmp
    return np.float32(XX)


In [155]:
'''project upper layer filter back to the input layer'''
from theano.sandbox.cuda.dnn import dnn_conv3d
def deconv(W0,W1):
    print(W0.shape)
    print(W1.shape)
    filter_size = W0.shape[2:5]
    num_filter = W0.shape[0]
    num_ch = W1.shape[0]
    W1_layer0 = dnn_conv3d(img=W1,kerns=W0.transpose(1,0,2,3,4),border_mode='valid',subsample=(1,1,1))
    W1_layer0 = np.asarray(W1_layer0.eval())
    print(W1_layer0.shape)
    return W1_layer0

In [151]:
'''visualize 3D data'''
def plot_cube(ax,x,y,z,a):
    "x y z location and alpha"
    
    ax.plot_surface([[x,x+1],[x,x+1]],[[y,y],[y+1,y+1]],z, alpha=a)
    ax.plot_surface([[x,x+1],[x,x+1]],[[y,y],[y+1,y+1]],z+1, alpha=a)

    ax.plot_surface(x,[[y,y],[y+1,y+1]],[[z,z+1],[z,z+1]], alpha=a)
    ax.plot_surface(x+1,[[y,y],[y+1,y+1]],[[z,z+1],[z,z+1]], alpha=a)

    ax.plot_surface([[x,x],[x+1,x+1]],y,[[z,z+1],[z,z+1]], alpha=a)
    ax.plot_surface([[x,x],[x+1,x+1]],y+1,[[z,z+1],[z,z+1]], alpha=a)

In [183]:
'''filter visualization'''
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

import tarfile
tarball = tarfile.open('shapenet.pkl', 'r')
ps = numpy.load(tarball.extractfile(tarball.getmember('_parameters')))
# sorted(ps.keys())

W0 = ps['|lenet|convolutionalsequence3|conv_0.W']
W1 = ps['|lenet|convolutionalsequence3|conv_1.W']
W1 = upsample(W1, 2)

W1_layer0 = deconv(W0,W1)
V = W1_layer0[10,0,:,:,:]
V = V/np.max(V) * 0.3 
V[V<0]=0

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(V.shape[0]):
    for j in range(V.shape[1]):
        for k in range(V.shape[2]):
            if V[i,j,k]!=0:
                plot_cube(ax,i,j,k,V[i,j,k])
plt.show()

(16, 1, 5, 5, 5)
(24, 16, 10, 10, 10)
(24, 1, 6, 6, 6)


In [171]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


def randrange(n, vmin, vmax):
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
n = 100
for c, m, zl, zh in [('r', 'o', -50, -25), ('b', '^', -30, -5)]:
    xs = randrange(n, 23, 32)
    ys = randrange(n, 0, 100)
    zs = randrange(n, zl, zh)
    ax.scatter(xs, ys, zs, c=c, marker=m)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [189]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

colorsMap='jet'
cm = plt.get_cmap(colorsMap)
cNorm = matplotlib.colors.Normalize(vmin=min([zs,zt]), vmax=max([zs,zt]))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xs =[1,2,3,4,5,6,7,8,9,10]
ys =[5,6,2,3,13,4,1,2,4,8]
zs =[2,3,3,3,5,7,9,11,9,10]

xt =[-1,-2,-3,-4,-5,-6,-7,8,-9,-10]
yt =[-5,-6,-2,-3,-13,-4,-1,2,-4,-8]
zt =[-2,-3,-3,-3,-5,-7,9,-11,-9,-10]

ax.scatter(xs, ys, zs, c=scalarMap.to_rgba(zs), marker='o')
ax.scatter(xt, yt, zt,  c=scalarMap.to_rgba(zt), marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

TypeError: float() argument must be a string or a number

In [193]:
c=scalarMap.to_rgba(.05)

TypeError: float() argument must be a string or a number